In [3]:
# ruff: noqa: F401, E402, T201, D103

from __future__ import annotations

import math
import sys
import time
from bisect import bisect_left, bisect_right
from collections import Counter, deque
from functools import cmp_to_key, lru_cache, reduce
from heapq import heapify, heappop, heappush, heappushpop, nsmallest
from importlib import reload
from itertools import (
    accumulate,
    chain,
    combinations,
    islice,
    pairwise,
    permutations,
    product,
    repeat,
    starmap,
    tee,
)
from math import comb, exp, factorial, inf, log, prod, sqrt
from operator import add, and_, contains, indexOf, itemgetter, mul, neg, or_, xor
from pprint import pprint
from typing import Callable, Iterable, List, Tuple

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from multiprocess import Process

import arrays
import combinatorics
import graphs
import lists
import mathematics
import matrix
import parsing
import search
import sequences
import sets
import stack
import strings
import trees

for m in (
    arrays,
    combinatorics,
    graphs,
    lists,
    mathematics,
    matrix,
    parsing,
    search,
    sequences,
    sets,
    stack,
    strings,
    trees,
):
    reload(m)


from combinatorics import fib
from search import lower_index, upper_index
from sequences import find_if
from strings import splint

print()


In [83]:
def just_add_operators(d: str, t: int) -> list:
    n = len(d)
    ret = []

    def split(i: int, expr: str, v: int, prev: int) -> list:
        if i >= n:
            if v == t:
                ret.append(expr)
            return

        for j in range(i + 1, min(n, i+1 if )):
            ns = d[i:j]
            nv = int(ns)
            split(j, expr + "+" + ns, v + nv, nv)
            split(j, expr + "-" + ns, v - nv, -nv)
            split(j, expr + "*" + ns, v - prev + prev * nv, prev * nv)

    for i in range(1, n + 1):
        ns = d[:i]
        nv = int(ns)
        split(i, ns, nv, nv)
    return ret


In [84]:
just_add_operators("232", 8)


['2+3*2', '2*3+2']

In [85]:
just_add_operators("123", 6)


['1+2+3', '1*2*3']

In [86]:
just_add_operators("3216", 19)


[]

In [87]:
just_add_operators("1054", 4)


['1*0*5+4']

In [88]:
just_add_operators("33", 14)


[]

In [89]:
just_add_operators("1", 1)


['1']